In [1]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("Binary Classification Metrics Test");
    var sc = new SparkContext(sparkConf);

    var MLUtils = require("eclairjs/mllib/MLUtils");
    var LogisticRegressionWithLBFGS = require("eclairjs/mllib/classification").LogisticRegressionWithLBFGS;
    var BinaryClassificationMetrics = require("eclairjs/mllib/evaluation/BinaryClassificationMetrics");
    var Vectors = require("eclairjs/mllib/linalg/Vectors");
    var LabeledPoint = require("eclairjs/mllib/regression/LabeledPoint");
    var Tuple = require('eclairjs/Tuple');

    var filename = ((typeof args !== "undefined") && (args.length > 1)) ?
        args[1] : "examples/data/mllib/sample_binary_classification_data.txt";

    var data = MLUtils.loadLibSVMFile(sc, filename);


Split data into training (60%) and test (40%)


In [2]:
    var split = data.randomSplit([0.6, 0.4], 11)
    var training = split[0].cache();
    var test = split[1];

    var model = new LogisticRegressionWithLBFGS()
        .setNumClasses(2)
        .run(training);

    var predictionAndLabels = test.mapToPair(function (lp, model, Tuple) {
        return new Tuple(model.predict(lp.getFeatures()), lp.getLabel());
    }, [model, Tuple]);

var result = new BinaryClassificationMetrics(predictionAndLabels);


 Precision by threshold


In [3]:
    var precision = result.precisionByThreshold();
    print("Precision by threshold: " + precision.collect());


 Recall by threshold


In [4]:
    var recall = result.recallByThreshold();
    print("Recall by threshold: " + recall.collect());


 F Score by threshold


In [5]:
    var f1Score = result.fMeasureByThreshold();
    print("F1 Score by threshold: " + f1Score.collect());

    var f2Score = result.fMeasureByThreshold(2.0);
    print("F2 Score by threshold: " + f2Score.collect());


 Precision-recall curve


In [6]:
    var prc = result.pr();
    print("Precision-recall curve: " + prc.collect());

    sc.stop();
